In [1]:
import sys
sys.path.append("../src")

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

import sys
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torchvision.datasets import FashionMNIST

import matplotlib.pyplot as plt
from pytorch_impl.nns import FCN
from pytorch_impl.nns.utils import warm_up_batch_norm, to_one_hot
from pytorch_impl.estimators import MatrixExpEstimator
from pytorch_impl import ClassifierTraining
from pytorch_impl.matrix_exp import matrix_exp, compute_exp_term

In [4]:
torch.manual_seed(0)

num_classes = 10

if torch.cuda.is_available() and False:
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu')

print('Torch version: {}'.format(torch.__version__))
print('Device: {}'.format(device))

train_loader = torch.utils.data.DataLoader(
    FashionMNIST(root='.', train=True, download=True,
          transform=transforms.ToTensor()),
    batch_size=32, shuffle=True, pin_memory=True)

test_loader = torch.utils.data.DataLoader(
    FashionMNIST(root='.', train=False, transform=transforms.ToTensor()),
    batch_size=32, shuffle=True, pin_memory=True)

device

Torch version: 1.3.1
Device: cpu


device(type='cpu')

In [5]:
a = 30 / 180 * np.pi
M = torch.tensor([[0, -1], [1, 0]]) * a
matrix_exp(M, device).cpu().numpy()

array([[ 0.8660254, -0.5      ],
       [ 0.5      ,  0.8660254]], dtype=float32)

In [6]:
a = 30 / 180 * np.pi
M = torch.tensor([[0, -1], [1, 0]]) * a
M_clone = M.clone().to(device)
(torch.matmul(M_clone, compute_exp_term(M, device)) + torch.eye(2)).numpy()

array([[ 0.8660254, -0.5      ],
       [ 0.5      ,  0.8660254]], dtype=float32)

In [12]:
D = 28

model     = FCN(1, D * D).to(device)
estimator = MatrixExpEstimator(model, num_classes, device, learning_rate=1., momentum=0.5)

In [13]:
_, (X, y) = next(enumerate(train_loader))
X, y = X.to(device), y.to(device)

estimator.predict(X)
for _ in range(20):
    estimator.fit(X, y)
    print(estimator.predict(X)[:5])

computing grads ... 0s
exponentiating kernel matrix ... 0s
tensor([[-0.9996, -1.0001, -1.0030, -0.9999, -0.9994,  0.9959, -0.9992, -0.9924,
         -1.0000, -1.0024],
        [-1.0011, -1.0012, -1.0057, -1.0007, -1.0025, -0.9986, -1.0011, -1.0015,
         -1.0007,  1.0023],
        [-0.9999,  0.9980, -1.0020, -0.9975, -1.0001, -1.0004, -0.9997, -1.0003,
         -0.9999, -1.0008],
        [-1.0004, -1.0005, -1.0012, -1.0002, -1.0006, -0.9990, -1.0001, -1.0006,
          0.9996, -1.0003],
        [-1.0003, -1.0007, -1.0017, -1.0000, -0.9997,  1.0032, -0.9990, -1.0040,
         -0.9990, -1.0012]])
computing grads ... 0s
exponentiating kernel matrix ... 0s
tensor([[-1.4998, -1.5001, -1.5017, -1.4999, -1.4997,  1.4978, -1.4996, -1.4959,
         -1.5000, -1.5013],
        [-1.5006, -1.5006, -1.5030, -1.5004, -1.5013, -1.4992, -1.5005, -1.5008,
         -1.5004,  1.5012],
        [-1.5000,  1.4990, -1.5010, -1.4987, -1.5000, -1.5002, -1.4999, -1.5002,
         -1.4999, -1.5004],
        [

computing grads ... 0s
exponentiating kernel matrix ... 0s
tensor([[-0.9945, -0.9944, -0.9943, -0.9945, -0.9945,  0.9946, -0.9945, -0.9948,
         -0.9944, -0.9944],
        [-0.9944, -0.9944, -0.9942, -0.9944, -0.9944, -0.9945, -0.9944, -0.9944,
         -0.9944,  0.9944],
        [-0.9944,  0.9945, -0.9944, -0.9945, -0.9944, -0.9944, -0.9945, -0.9944,
         -0.9944, -0.9944],
        [-0.9944, -0.9944, -0.9944, -0.9944, -0.9944, -0.9945, -0.9944, -0.9944,
          0.9945, -0.9944],
        [-0.9944, -0.9944, -0.9944, -0.9944, -0.9945,  0.9943, -0.9945, -0.9943,
         -0.9945, -0.9944]])
computing grads ... 0s
exponentiating kernel matrix ... 0s
tensor([[-0.9973, -0.9973, -0.9973, -0.9973, -0.9973,  0.9972, -0.9973, -0.9971,
         -0.9973, -0.9974],
        [-0.9973, -0.9973, -0.9975, -0.9973, -0.9974, -0.9972, -0.9973, -0.9973,
         -0.9973,  0.9974],
        [-0.9973,  0.9972, -0.9974, -0.9972, -0.9973, -0.9973, -0.9973, -0.9973,
         -0.9973, -0.9973],
        [